# Defining Release Scenarios

The whole point of `GasDispersion.jl` is to model chemical releases into the atmosphere, to do this one needs to have a defined chemical release scenario. 

The default is to assume the release is a jet of material issuing from a single point, emitting a constant mass emission rate for a given release duration. The coordinate system is such that the release point is at *x=0*, *y=0*, *z=h* where *x* is the downwind distance, *y* the cross wind distance, and *z* the elevation. 

<a title="Mbeychok, CC BY-SA 3.0 &lt;http://creativecommons.org/licenses/by-sa/3.0/&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Gaussian_Plume.png"><img alt="Gaussian Plume" src="https://upload.wikimedia.org/wikipedia/commons/1/10/Gaussian_Plume.png"></a>

<a href="https://commons.wikimedia.org/wiki/File:Gaussian_Plume.png">Mbeychok</a>, <a href="http://creativecommons.org/licenses/by-sa/3.0/">CC BY-SA 3.0</a>, via Wikimedia Commons

The gas dispersion models require the following parameters to be defined for a given scenario:

- mass emission rate, kg/s
- release duration, s
- jet diameter, m
- jet velocity, m/s
- jet density, kg/m³
- release pressure, Pa
- release temperature, K
- release height, m
- windspeed, m/s
- ambient density, kg/m³
- ambient pressure, Pa
- ambient temperature, K
- pasquill stability class

Standard methods given:
+ *Lee's Loss Prevention in the Process Industries*
+ CCPS *Guidelines for Consequence Analysis of Chemical Releases*
+ CCPS *Guidelines for Chemical Process Quantitative Risk Analysis*
+ CCPS *Guidelines for Vapour Cloud Explosion, Pressure Vessel Burst, BLEVE and Flash Fire Hazards*

and many other texts on chemical process safety can be used to determine all of these parameters and a `Scenario` created directly.

In [2]:
using GasDispersion

In [3]:
release = Release(mass_rate=1.0,
                  duration=10.0,
                  diameter=0.25,
                  velocity=15.67,
                  height=1.0,
                  pressure=101325,
                  temperature=450,
                  density=1.3)

scenario = Scenario(release,Ambient())

Atmospheric conditions:
    pressure: 101325 Pa 
    temperature: 298.15 K 
    density: 1.225 kg/m^3 
    windspeed: 1.5 m/s 
    stability: F  
    
Release conditions:
    mass_rate: 1.0 kg/s 
    duration: 10.0 s 
    diameter: 0.25 m 
    velocity: 15.67 m/s 
    height: 1.0 m 
    pressure: 101325 Pa 
    temperature: 450 K 
    density: 1.3 kg/m^3 
    

`GasDispersion` also comes with some basic tools for constructing simple release scenarios for common situations

## Hole Discharge
The simplest release case is a release from a hole, where the hole is treated as a circular orifice of negligible length.

### Liquid Phase
A liquid jet through a small hole can be modeled by the Bernoulli equation, with the jet velocity given by

$$ u_j = c_d  \sqrt{ 2 \left( P_1 - P_2 \over \rho_l \right) } $$

where
- $u_j$ is the jet velocity, m/s
- $c_d$ the discharge coefficient, unitless
- $P_1$ the pressure upstream of the hole
- $P_2$ the ambient pressure, Pa, i.e. the pressure outside the hole
- $\rho_l$ the liquid density, kg/m³

the mass emission rate is then given by
$$ \dot{m} = \rho_l u_j A$$

where $A$ is the cross-sectional area of the hole, in m²

#### Example

This scenario is adapted from CCPS *Guidelines for Consequence Analysis of Chemical Releases*, CCPS, pg 40.

Suppose a leak of a liquid from a storage tank through a hole. The tank headspace is at 0.1 barg and there is 2 m of liquid head above the hole. The liquid has a density of 490 kg/m³, and is in thermal equilibrium with the environment. Assume a circular hole with a discharge coefficient of 0.63 and a diameter of 1cm.

For ambient conditions we assume the atmosphere is at standard conditions of 1atm and 25°C, with a windspeed of 1.5m/s and class F stability.

Assumptions:
- discharge coefficient, $c_d = 0.63 $
- diameter of the hole, $d = 0.010 \textrm{m}$
- liquid density, $\rho_l = 490 \textrm{kg/m}^3$
- liquid temperature, $T_1 = 298.15 \textrm{K}$
- ambient pressure, $P_2 = 101325 \textrm{Pa}$
- ambient temperature, $T_2 = 298.15 \textrm{K}$
- density of air at ambient conditions, $\rho_a = 1.225 \textrm{kg/m}^3$
- ambient windspeed, $u = 1.5 \textrm{m/s}$

In [4]:
# What we know
c_d = 0.63
d = 0.010    #m
ρₗ = 490.0    #kg/m³
T₁ = 298.15   #K
P₂ = 101325.0 #Pa
T₂ = 298.15   #K
ρₐ = 1.225    #kg/m³
u  = 1.5      #m/s
stability = "F"

"F"

What we need to solve for is
- $P_r$, the release pressure, Pa,
- $u_j$, the jet velocity, m/s
- $\dot{m}$, the mass emission rate, kg/s

The pressure immediately inside the liquid discharge is the sum of the headspace pressure and the pressure due to the 2 m of liquid head and is

$$ P_r = P_{headspace} + \rho_j g h $$
$$ P_r = (10000 Pa + 101325 Pa) + 490 kg/m³ \cdot 9.80616 m/s² \cdot 2 m $$

In [5]:
g  = 9.80616  # gravitational acceleration, m/s²

P₁ = (10000 + P₂) + ρₗ*g*2

120935.0368

The jet velocity comes from the Bernoulli equation

$$ u_j = c_d  \sqrt{ 2 \left( P_1 - P_2 \over \rho_l \right) } $$

In [6]:
uⱼ = c_d * √( 2*( (P₁ - P₂) / ρₗ) )

5.636333880812954

We are assuming a circular hole so the cross sectional area is just

$$ A = {\pi \over 4} d^2 $$

In [7]:
A = (π/4)*d^2

7.853981633974483e-5

and the mass emission rate is given by the equation
$$ \dot{m} = \rho_l u_j A$$

In [8]:
m = ρₗ*uⱼ*A

0.21691154763598

we can now put this all together to make a release scenario, ready for modeling:

In [9]:
release = Release(mass_rate=m,
                  duration=Inf,
                  diameter=d,
                  velocity=uⱼ,
                  height=1,
                  pressure=P₂,
                  temperature=T₁,
                  density=ρₗ)

atmosphere = Ambient(pressure=P₂,
                     temperature=T₂,
                     density=ρₐ,
                     windspeed=u,
                     stability="F")

s1 = Scenario(release,atmosphere)

Atmospheric conditions:
    pressure: 101325.0 Pa 
    temperature: 298.15 K 
    density: 1.225 kg/m^3 
    windspeed: 1.5 m/s 
    stability: F  
    
Release conditions:
    mass_rate: 0.21691154763598 kg/s 
    duration: Inf s 
    diameter: 0.01 m 
    velocity: 5.636333880812954 m/s 
    height: 1 m 
    pressure: 101325.0 Pa 
    temperature: 298.15 K 
    density: 490.0 kg/m^3 
    

The helper function `scenario_builder` does basically this, which simplifies the workflow immensly. All one needs to do is supply some basic information and, using default values for ambient conditions, a release scenario is generated.

In [10]:
source=JetSource(phase=:liquid, dischargecoef=c_d, diameter=d,
            pressure=P₁, temperature=T₁, density=ρₗ,height=1)

s2 = scenario_builder(source, atmosphere)

Atmospheric conditions:
    pressure: 101325.0 Pa 
    temperature: 298.15 K 
    density: 1.225 kg/m^3 
    windspeed: 1.5 m/s 
    stability: F  
    
Release conditions:
    mass_rate: 0.21691154763598 kg/s 
    duration: Inf s 
    diameter: 0.01 m 
    velocity: 5.636333880812954 m/s 
    height: 1 m 
    pressure: 101325.0 Pa 
    temperature: 298.15 K 
    density: 490.0 kg/m^3 
    

In [11]:
s1 ≈ s2

true

### Gas Phase

A gas jet release, where the gas is ideal and the expansion through the jet is an isentropic process, can be modeled using a modified Bernoulli equation, with

$$ G = \rho u = c_d \sqrt{ \rho_1 P_1 \left( 2 k \over k-1 \right) \left[ \left(P_2 \over P_1\right)^{2 \over k} - \left(P_2 \over P_1\right)^{k+1 \over k} \right]} $$

for non-choked flow and

$$ G = c_d \sqrt{ \rho_1 P_1 k \left( 2 \over k+1 \right)^{k+1 \over k-1} } $$

for choked flow, which occurs when

$$ \left(P_2 \over P_1 \right) \lt \left( 2 \over k+1 \right)^{k \over k-1} $$

where
- $G$ is the mass velocity, kg/m²s
- $c_d$ the discharge coefficient, unitless
- $k$ is the ratio of heat capacities, cp/cv, unitless
- $P_1$ the pressure upstream of the hole
- $\rho_1$ the gas density upstream of the hole, kg/m³
- $P_2$ the ambient pressure, Pa, i.e. the pressure outside the hole

the mass emission rate is then given by
$$ \dot{m} = G A$$

where $A$ is the cross-sectional area of the hole, in m²

The properties of the jet immediately at the exit of the orifice are then given by
$$ {\rho_o \over \rho_1} = \left( P_o \over P_1 \right)^{1 \over k} $$
$$ {T_o \over T_1} = \left( P_o \over P_1 \right)^{k-1 \over k}$$

#### Example

This scenario is adapted from CCPS *Guidelines for Consequence Analysis of Chemical Releases*, CCPS, pg 47

Suppose a leak of propane through a 1cm hole with a temperature of 25C and pressure of 4barg. The hole is 1m above the ground and has a discharge coefficient of 0.85. The heat capacity ratio for propane is $k=1.15$ The atmosphere is otherwise at ambient conditions.

Assumptions:
- discharge coefficient, $c_d = 0.85 $
- heat capacity ratio, $k = 1.15$
- molar weight, $Mw = 44.097 \textrm{kg/kmol}$
- diameter of the hole, $d = 0.010 \textrm{m}$
- vessel temperature, $T_1 = 298.15 \textrm{K}$
- vessel pressure, $P_1 = 4 \textrm{barg}$
- ambient pressure, $P_2 = 101325 \textrm{Pa}$
- ambient temperature, $T_2 = 298.15 \textrm{K}$
- density of air at ambient conditions, $\rho_a = 1.225 \textrm{kg/m}^3$
- ambient windspeed, $u = 1.5 \textrm{m/s}$

In [12]:
c_d = 0.85
k   = 1.15
Mw  = 44.097
d   = 0.010

# ambient conditions
P₂  = 101325
T₂  = 298.15
ρₐ  = 1.225
u   = 1.5

1.5

First we determine the absolute pressure and gas density at point *1* (using the ideal gas law)

$$ \rho_1 = { Mw P_1 \over R T_1 } $$

In [13]:
R  = 8.31446261815324 #m³⋅kPa⋅K⁻¹⋅kmol⁻¹

T₁ = T₂
P₁ = 4e5 + P₂
ρ₁ = (Mw*P₁)/(R*T₁)/1000

8.917834500965851

We check if the flow is sonic (choked)
$$ \left(P_2 \over P_1 \right) \lt \left( 2 \over k+1 \right)^{k \over k-1} $$

In [14]:
(P₂/P₁) < (2/(k+1))^(k/(k-1))

true

Therefore the pressure at the orifice is given by
$$ {P_o \over P_1} = \eta = \left( 2 \over k+1 \right)^{k \over k-1} $$

In [15]:
η  = (2/(k+1))^(k/(k-1))
Pₒ = η*P₁

287952.6877282304

The mass velocity is given by
$$ G = c_d \sqrt{ \rho_1 P_1 k \left( 2 \over k+1 \right)^{k+1 \over k-1} } $$

In [16]:
G = c_d * √(ρ₁*P₁*k*(2/(k+1))^((k+1)/(k-1)) )

1147.7921164988413

From which we can calculate the mass discharge rate and velocity

In [17]:
A  = (π/4)*d^2
m  = G*A

0.090147382026026

In [18]:
ρₒ = ρ₁*η^(1/k)
uₒ = G/ρₒ

208.4460121106216

In [19]:
Tₒ = T₁*η^((k-1)/k)

277.3488372093023

we can now put this all together to make a release scenario, ready for modeling:

In [20]:
release = Release(mass_rate=m,
                  duration=Inf,
                  diameter=d,
                  velocity=uₒ,
                  height=1,
                  pressure=Pₒ,
                  temperature=Tₒ,
                  density=ρₒ)

atmosphere = Ambient(pressure=P₂,
                     temperature=T₂,
                     density=ρₐ,
                     windspeed=u,
                     stability="F")

s1 = Scenario(release,atmosphere)

Atmospheric conditions:
    pressure: 101325 Pa 
    temperature: 298.15 K 
    density: 1.225 kg/m^3 
    windspeed: 1.5 m/s 
    stability: F  
    
Release conditions:
    mass_rate: 0.090147382026026 kg/s 
    duration: Inf s 
    diameter: 0.01 m 
    velocity: 208.4460121106216 m/s 
    height: 1 m 
    pressure: 287952.6877282304 Pa 
    temperature: 277.3488372093023 K 
    density: 5.506423965020313 kg/m^3 
    

The helper function `scenario_builder` does basically this, which simplifies the workflow immensly. All one needs to do is supply some basic information and, using default values for ambient conditions, a release scenario is generated.

In [21]:
source=JetSource(phase=:gas, dischargecoef=c_d, k=1.15, diameter=d,
            pressure=P₁, temperature=T₁, density=ρ₁, height=1)

s2 = scenario_builder(source, atmosphere)

Atmospheric conditions:
    pressure: 101325 Pa 
    temperature: 298.15 K 
    density: 1.225 kg/m^3 
    windspeed: 1.5 m/s 
    stability: F  
    
Release conditions:
    mass_rate: 0.09014738202602605 kg/s 
    duration: Inf s 
    diameter: 0.01 m 
    velocity: 208.44601211062175 m/s 
    height: 1 m 
    pressure: 287952.6877282304 Pa 
    temperature: 277.3488372093023 K 
    density: 5.506423965020313 kg/m^3 
    

In [22]:
s1 ≈ s2

true